# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [24]:
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset

In [ ]:
model_name = "distilbert-base-uncased"

In [26]:
# Load the pretrained model 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Move the model to the GPU if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)


OSError: gpt-2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Load and prepare the dataset
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [ ]:

# Load and prepare the datase
# Let's use a lambda function to tokenize all the examples
tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_ds["train"]

# Prepare inputs for the model
inputs = tokenizer(tokenized_ds["train"]["sms"], padding=True, truncation=True, return_tensors="pt")


In [ ]:
# Set the model to evaluation mode
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [ ]:
def evaluate_model(model, tokenized_dataset):
    # Move the model to evaluation mode
    model.eval()
    
    # Prepare inputs and move them to the correct device
    # inputs = tokenizer(tokenized_dataset["test"]["sms"], padding=True, truncation=True, return_tensors="pt").to(device)
    inputs = tokenizer(tokenized_dataset["test"]["sms"], padding=True, truncation=True, return_tensors="pt")
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    predictions = np.argmax(logits.cpu().numpy(), axis=1)  # Move logits to CPU for numpy operations
    true_labels = tokenized_dataset["test"]["label"]
    accuracy = (predictions == true_labels).mean()
    return accuracy

pretrained_accuracy = evaluate_model(model, tokenized_ds)
print(f"Accuracy before fine-tuning: {pretrained_accuracy}")

Accuracy before fine-tuning: 0.3201793721973094


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [ ]:
from peft import LoraConfig, get_peft_model
# Configuración de LoRA
config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # Alpha
    lora_dropout=0.1,  # Dropout rate
    bias="none",  # Bias setting
    target_modules=["classifier"]  # Specify the target modules
)

In [ ]:

lora_model = get_peft_model(model, config)

lora_model.print_trainable_parameters()


trainable params: 12,320 || all params: 66,967,330 || trainable%: 0.0184


In [ ]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./spam_not_spam",
        # Set the learning rate
        learning_rate=2e-3,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2, 
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)





/home/ric/.cache/pypoetry/virtualenvs/genai-YY3L6q6E-py3.12/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_13835/2370546731.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


  0%|          | 0/4460 [00:00<?, ?it/s]

{'loss': 0.2572, 'grad_norm': 0.17049166560173035, 'learning_rate': 0.0017757847533632287, 'epoch': 0.22}
{'loss': 0.0787, 'grad_norm': 0.03978770598769188, 'learning_rate': 0.0015515695067264575, 'epoch': 0.45}
{'loss': 0.0757, 'grad_norm': 0.00702950032427907, 'learning_rate': 0.001327354260089686, 'epoch': 0.67}
{'loss': 0.0858, 'grad_norm': 0.05223963037133217, 'learning_rate': 0.001103139013452915, 'epoch': 0.9}


  0%|          | 0/558 [00:00<?, ?it/s]

{'eval_loss': 0.07081382721662521, 'eval_accuracy': 0.9802690582959641, 'eval_runtime': 2.3408, 'eval_samples_per_second': 476.332, 'eval_steps_per_second': 238.38, 'epoch': 1.0}
{'loss': 0.0799, 'grad_norm': 1.7423319816589355, 'learning_rate': 0.0008789237668161434, 'epoch': 1.12}
{'loss': 0.0872, 'grad_norm': 2.160661694006194e-07, 'learning_rate': 0.0006547085201793722, 'epoch': 1.35}
{'loss': 0.0735, 'grad_norm': 2.314220666885376, 'learning_rate': 0.00043049327354260095, 'epoch': 1.57}
{'loss': 0.0576, 'grad_norm': 0.0040724980644881725, 'learning_rate': 0.00020627802690582962, 'epoch': 1.79}


  0%|          | 0/558 [00:00<?, ?it/s]

{'eval_loss': 0.0568753145635128, 'eval_accuracy': 0.9865470852017937, 'eval_runtime': 2.521, 'eval_samples_per_second': 442.291, 'eval_steps_per_second': 221.344, 'epoch': 2.0}
{'train_runtime': 34.5328, 'train_samples_per_second': 258.248, 'train_steps_per_second': 129.153, 'train_loss': 0.10133529415045084, 'epoch': 2.0}


TrainOutput(global_step=4460, training_loss=0.10133529415045084, metrics={'train_runtime': 34.5328, 'train_samples_per_second': 258.248, 'train_steps_per_second': 129.153, 'total_flos': 78977267223744.0, 'train_loss': 0.10133529415045084, 'epoch': 2.0})

In [ ]:
trainer.evaluate()
model_save_path = "./my_gpt2"  # Specify the directory where you want to save the model
lora_model.save_pretrained(model_save_path)

  0%|          | 0/558 [00:00<?, ?it/s]

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
from peft import AutoPeftModelForCausalLM

lora_model = AutoPeftModelForCausalLM.from_pretrained("./my_model")
# lora_model.to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
fine_tuned_accuracy = evaluate_model(lora_model, tokenized_ds)
print(f"Accuracy after fine-tuning: {fine_tuned_accuracy}")


Accuracy after fine-tuning: 0.8708520179372198


In [ ]:

# Compare results
print(f"Accuracy improvement: {fine_tuned_accuracy - pretrained_accuracy}")

Accuracy improvement: 0.5506726457399104
